![Département de Mathématiques](https://ktzanev.github.io/logolabopp/math-ulille/math-ulille_100.gif)

# TP 2 - Les équations différentielles de dimension 1 (deuxième partie)

Dans ce deuxième TP nous continuons l'étudier numériquement des équations du premier ordre de dimension 1.<br>
Il s'agit d'un exemple vu dans les fiches A de TD, ainsi que d'un exercice du S2 de 2021.

**Attention :** Pour que le code entre les exercices soit au maximum similaire et permettre de faire des copier/coller, nous utilisons plusieurs fois les mêmes noms pour des fonctions différentes. **Ceci est une très mauvaise pratique qui mène à des résultats imprévisibles si on n'exécute pas les cellules dans l'ordre.**

On commence par charger les bibliothèques « standards » dont on aura besoin.

In [ ]:
# numpy pour les calculs (vectoriels)
import numpy as np
# matplotlib (référencé comme `plt` ici) pour les graphiques
from matplotlib import pyplot as plt
# la bibliothèque pour tracer les trajectoires
from scipy.integrate import odeint
# la bibliothèque qui permet d'utiliser (le décorateur) @interact
from ipywidgets import interact

## Une fonction utile

On reprend du TP1 la fonction `champ_normalise` qui permet de tracer le champ de vecteur $(1,f(y,t))$ **normalisé** associé au problème de Cauchy $y'=f(y,t)$.

In [ ]:
# affiche le champ normalisé sur la figure courante
# - f représente la fonction de y'=f(y,t)
# - tmin, tmax, ymin, ymax est la fenêtre d'affichage
# - N est le nombre de discrétisations en chaque direction
def champ_normalise(f, tmin, tmax, ymin, ymax, N=15, **kw):
    F = lambda Y: [np.ones_like(Y[0]), f(Y[1], Y[0])]  # la fonction du problème autonome associé
    T = np.linspace(tmin, tmax, N)  # abscisses des points de la grille
    Y = np.linspace(ymin, ymax, N)  # ordonnées des points de la grille
    U, V = F(np.meshgrid(T, Y))  # les composantes du champ de vecteurs
    M = np.hypot(U, V)  # calcule la norme du vecteur (U,V)
    M[M == 0] = 1  # évite la division par 0
    U /= M  # normalise la composante U
    V /= M  # normalise la composante V
    return plt.quiver(T, Y, U, V, angles='xy', **kw)  # trace le champ de vecteurs sur la grille de NxN points

## exercice 11


Notre équation autonome étant
$$
    y'=\frac{y}{2}\big( 1-\frac{3}{y^2+2}\big)
$$
la fonction $f$ à définir est $f(y,t) =\frac{y}{2}\big( 1-\frac{3}{y^2+2}\big)$.

In [ ]:
# la fonction f

# test
assert f(0, 0) == 0, "f(0,*) = 0"
assert f(1, 1) == 0, "f(1,*) = 0"
assert np.shape(f(np.ones(3), np.zeros(3))) == (3, ), "la fonction doit accepter des vecteurs comme paramètres"

### La représentation graphique

Nous avons vu en TD que ce problème possède trois solutions constantes : $1$, $0$ et $-1$. Et que entre ces constantes la solution est monotone, car la dérivée est de signe constant.

Adapter les représentations graphiques des exercices du TP1 à cet exercice, représentant les solutions constantes ainsi que la solution qui vérifie $y(t_0) = y_0$.

*Aide pour la question d'après :* Si possible, représenter sur l'ordonné l'orbite $y(\mathbb{R})\subset \mathbb{R}$ décrite par cette solution.

In [ ]:
# choix de la fenêtre
tmin, tmax, ymin, ymax = -10, 10, -5, 5



**Question :** En observant le graphique, l'ensemble $\mathbb{R}$ est la réunion de combien d'orbites ?

## exercice 1 du DS2 (2021)

On considère le problème de Cauchy, avec $|\cdot|$ qui est la valeur absolue sur $\mathbb{R}$ :

$$
    \begin{cases}
        y'(t)=|y(t)+t|\\
        y(0)=0
    \end{cases}
$$

1) Établir que ce problème admet une unique solution $y$ de classe $C^1$ globale, i.e. définie sur $\mathbb{R}$ tout entier.

2) Calculer cette solution.

## La solution globale
Définir la fonction `s` qui est la solution globale vérifiant `s(0)==0`. Cette fonction doit pouvoir prendre comme argument des `np.array`.

In [ ]:
# test
assert s(0) == 0, "La fonction doit vérifier la condition initiale s(0) == 0"
assert s(-2) < s(1), "La fonction doit être strictement croissante"
assert np.shape(s(np.ones(3))) == (3, ), "la fonction doit accepter des vecteurs comme paramètres"

## La représentation graphique

Adapter les représentations graphiques dynamiques des exercices du TP1 à cet exercice, représentant la solution globale `s` ainsi que la solution trouvée par `odeint` qui vérifie $y(t_0) = y_0$.

In [ ]:
# Définition de la fonction f(y,t) de l'équation y'=f(y,t)

# test
assert f(0, 0) == 0, "f(0,0) = 0"
assert f(1, -1) == 0, "f(1,-1) = 0"
assert np.shape(f(np.ones(3), np.zeros(3))) == (3, ), "la fonction doit accepter des vecteurs comme paramètres"

In [ ]:
# choix de la fenêtre
tmin, tmax, ymin, ymax = -2, 2, -2, 2

# choix de la condition initiale grâce à des curseurs
@interact(t_0=(tmin, tmax, .5), y_0=(ymin, ymax, .25))
# le dessin pour la condition initiale (t_0,y_0)
def dyn_solution(t_0=0, y_0=0):
    # taille de la figure
    plt.figure(figsize=(2 * (tmax - tmin), 2 * (ymax - ymin)))
    # on détermine la fenêtre d'affichage
    plt.axis([tmin, tmax, ymin, ymax])
    # on trace les axes en `yellow`
    plt.plot([tmin, tmax], [0, 0], "y")
    plt.plot([0, 0], [ymin, ymax], "y")

    # on dessine le champ normalisé
    champ_normalise(f, tmin, tmax, ymin, ymax, width=.002)

    # la solution que nous avons trouvée en TD pour (0,0)
    t = np.linspace(tmin, tmax, 100)
    plt.plot(t, s(t), "r")

    # La solution approchée, trouvée avec odeint
    # la partie de la courbe pour t dans [t_0,tmax] (le futur)
    futur = np.linspace(t_0, tmax, 50)
    v = odeint(f, y_0, futur)
    plt.plot(futur, v, "b")
    # la partie de la courbe pour t dans [tmin,t_0] (le passé)
    passe = np.linspace(t_0, tmin, 50)
    v = odeint(f, y_0, passe)
    plt.plot(passe, v, "g--")

    # le point de départ (la condition initiale)
    plt.plot(t_0, y_0, 'or')
    # le titre
    plt.title(f"Solution avec y({t_0})={y_0}.")
    # et on affiche tout ça
    plt.show()

# --- solution ---

## exercice 3 du DS2 (2021)

On va terminer ce TP par un exercice dont les solutions maximales ne sont pas globales, car elles explosent en temps fini.

Soit $y$ la solution maximale du problème de Cauchy :

$$
    \begin{cases}
        y'(t)=y(t)+y^3(t)\\
        y(0)=1
    \end{cases}
$$

On peut voir facilement que ce problème vérifie les conditions de Cauchy-Lipschitz local, mais pas global. Et comme la fonction constante $0$  est une solution (globale) de cette équation autonome, on conclut que les autres solutions ne s'annulent pas.
Puis en posant $z(t)= \frac{1}{y^2(t)}$ et voit que $z$ est une solution de $z' = -2 -2z$ et pour finir on trouve que la solution maximale de l'équation de départ est $y(t)= \frac{1}{\sqrt{2e^{-2t}-1}}$ pour $t\in]-\infty,\log(\sqrt{2})[$.

Essayon de représenter ceci graphiquement, comme dans les exercices précédents.

In [ ]:
# Définition de la fonction f(y,t) de l'équation y'=f(y,t)
f = lambda y, t: y+y**3

# test
assert f(0, 0) == 0, "f(0,*) = 0"
assert f(1, -1) == 2, "f(1,*) = 1"
assert np.shape(f(np.ones(3), np.zeros(3))) == (3, ), "la fonction doit accepter des vecteurs comme paramètres"

In [ ]:
s = lambda t : 1/np.sqrt(2*np.exp(-2*t)-1)

# test
assert s(0) == 1, "la condition initiale est s(0) = 1"
assert np.shape(s(np.zeros(3))) == (3, ), "la fonction doit accepter des vecteurs comme paramètres"

In [ ]:
# choix de la fenêtre
tmin, tmax, ymin, ymax = -2, 2, -2, 2

# choix de la condition initiale grâce à des curseurs
@interact(t_0=(tmin, tmax, .5), y_0=(ymin, ymax, .25))
# le dessin pour la condition initiale (t_0,y_0)
def dyn_solution(t_0=0, y_0=0):
    # taille de la figure
    plt.figure(figsize=(2 * (tmax - tmin), 2 * (ymax - ymin)))
    # on détermine la fenêtre d'affichage
    plt.axis([tmin, tmax, ymin, ymax])
    # on trace les axes en `yellow`
    plt.plot([tmin, tmax], [0, 0], "y")
    plt.plot([0, 0], [ymin, ymax], "y")

    # on dessine le champ normalisé
    champ_normalise(f, tmin, tmax, ymin, ymax, width=.002)

    # la solution que nous avons trouvéepour t < log(sqrt(2))
    t = np.linspace(tmin, np.log(np.sqrt(2))-1e-10, 100)
    plt.plot(t, s(t), "r")

    # La solution approchée, trouvée avec odeint
    # la partie de la courbe pour t dans [t_0,tmax] (le futur)
    futur = np.linspace(t_0, tmax, 50)
    v = odeint(f, y_0, futur)
    plt.plot(futur, v, "b")
    # la partie de la courbe pour t dans [tmin,t_0] (le passé)
    passe = np.linspace(t_0, tmin, 50)
    v = odeint(f, y_0, passe)
    plt.plot(passe, v, "g--")

    # le point de départ (la condition initiale)
    plt.plot(t_0, y_0, 'or')
    # le titre
    plt.title(f"Solution avec y({t_0})={y_0}.")
    # et on affiche tout ça
    plt.show()

On constate que les solutions trouvées par `odeint` explosent numériquement. Une façon « d'arranger » ça est de « ralentir » `f` en dehors de la fenêtre d'affichage. Pour cela on définit une fonction `ralentir` qui modifie `f` en dehors de la fenêtre visible pour éviter les explosions en temps « courts ».

In [ ]:
# cutoff une fonction C^0 qui :
#  - est égale à 1 pour |Y| < r
#  - décroit exponentiellement en dehors de ce disque
def cutoff(Y, r):
    nY = np.maximum(Y * Y - r * r, 0)
    return np.exp(-nY)

# la fonction qui « ralentie » F en dehors de la fenêtre [xmin, xmax, ymin, ymax]
def ralentir(f, ymin, ymax):
    r = max(abs(ymax), abs(ymin))  # [-r,r] contient [ymin,ymax]
    return lambda Y, t: f(Y,t) * cutoff(Y, r)

On reprend le code précédent, mais on remplace `f` par `F = ralentir(f, ymin, ymax)` dans `odeint`. Et ainsi on évite les explosions numériques.

In [ ]:
# choix de la fenêtre
tmin, tmax, ymin, ymax = -2, 2, -2, 2

# choix de la condition initiale grâce à des curseurs
@interact(t_0=(tmin, tmax, .5), y_0=(ymin, ymax, .25))
# le dessin pour la condition initiale (t_0,y_0)
def dyn_solution(t_0=0, y_0=0):
    # taille de la figure
    plt.figure(figsize=(2 * (tmax - tmin), 2 * (ymax - ymin)))
    # on détermine la fenêtre d'affichage
    plt.axis([tmin, tmax, ymin, ymax])
    # on trace les axes en `yellow`
    plt.plot([tmin, tmax], [0, 0], "y")
    plt.plot([0, 0], [ymin, ymax], "y")

    # on dessine le champ normalisé
    champ_normalise(f, tmin, tmax, ymin, ymax, width=.002)

    # la solution que nous avons trouvéepour t < log(sqrt(2))
    t = np.linspace(tmin, np.log(np.sqrt(2))-1e-10, 100)
    plt.plot(t, s(t), "r")

    F = ralentir(f, ymin, ymax)
    # La solution approchée, trouvée avec odeint
    # la partie de la courbe pour t dans [t_0,tmax] (le futur)
    futur = np.linspace(t_0, tmax, 50)
    v = odeint(F, y_0, futur)
    plt.plot(futur, v, "b")
    # la partie de la courbe pour t dans [tmin,t_0] (le passé)
    passe = np.linspace(t_0, tmin, 50)
    v = odeint(F, y_0, passe)
    plt.plot(passe, v, "g--")

    # le point de départ (la condition initiale)
    plt.plot(t_0, y_0, 'or')
    # le titre
    plt.title(f"Solution avec y({t_0})={y_0}.")
    # et on affiche tout ça
    plt.show()